# Data Clean up

Haytham Mohamed - INFS890 Spring 2020

This notebook is to check missing data and outliers

In [174]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn import preprocessing
from datetime import datetime
from scipy import stats

sns.set()
sns.set(color_codes=True)
#sns.set_color_codes()

pd.options.display.max_rows = 15
pd.options.display.float_format = '{:,.3f}'.format

# Read the raw data. 

All data is in one file, where each metric is listed under each other with three columns: 

1- metric: The name of the metric

2- date: the data and time of the metric

3- value: the metric value


In [175]:
# 1- read processed file

home_dir = '/Users/hmohamed/github/data-research-spring2020/sock-shop'

file_dir = home_dir + '/metrics-1pod/'
processed_dir = home_dir + '/processed-data/' 


orders_ltcy_file = '1_orders_ltcy.csv'
carts_ltcy_file = '2_carts_ltcy.csv'
 
carts_req_rate_file = '3_carts_req.csv'
orders_req_rate_file = '4_orders_req.csv'

service_cpu_usage_file = '6_svc_cpu_use.csv'
service_cpu_sat_file = '8_svc_cpu_sat.csv'

nodes_cpu_usage_file = '7_nodes_cpu_use.csv'
nodes_cpu_sat_file = '9_nodes_cpu_sat.csv'

services_pods_file = '5_svc_pods.csv'


service_input_files = [service_cpu_usage_file
                      #, service_cpu_sat_file
                      #, system_cpu_usage_file
                      #, system_cpu_sat_file
                       
                      #, orders_ltcy_file
                      #, carts_ltcy_file
                     ]

concated_data_file = 'all_data.csv'

save=True
frequency = '15S'  # S for second , T for minute

In [176]:
def read_df(file_dir, data_file):
    df = pd.read_csv(file_dir + data_file, header=None, skiprows=1, names=['unit', 'metric', 'date', 'value'])
    #df = df.drop(['pods'], axis=1)
    return to_time_series(df)

def to_time_series(df, index_col_name='date', value_col_name='value'):
    df[index_col_name] = pd.to_datetime(df[index_col_name])
    df[value_col_name] = pd.to_numeric(df[value_col_name])
    df.set_index(index_col_name, inplace=True)
    df.sort_index(inplace=True)
    return df
    

# This function merges and alines the metrics timeseries data into a data frame, a column for every feature
def expand(df, by_col='unit', base=37, dup='mean'):
    # first convert to time series
    #df = toTimeSeries(df, 'date')
    by_col_values = np.unique(df[by_col])
    metrics_df = pd.DataFrame()
    i = 0
    for col_value in by_col_values:
        #print("Processing metric for column: %", col_value)
        series = extract_metric_series(df, by_col, col_value)
        series = resample(series, value_column_name=col_value, dup=dup, base=base)  
        if i == 0:
            metrics_df = series
        else:
            metrics_df = merge(metrics_df, series)
        i = i + 1
    return metrics_df 

# T for minutes, S for seconds
# remedy duplicates by either taking the maximum (=max) or average  (=mean) them
def resample(df, value_column_name, index_col_name='date', frequency = frequency, interpolate = True
             , interpolate_method = 'linear', base=37, dup = 'mean'):
    # eliminate dups in timestamp
    if dup == 'max':
        df = df.groupby([index_col_name])[value_column_name].max()   # taking max
        df = pd.DataFrame(df)
    elif dup == 'sum':
        df = df.groupby([index_col_name])[value_column_name].sum()
    else:
        df = df.groupby([index_col_name]).mean()   # taking mean
        
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)  # order the timeseries
    
    # fill in missing interval (upsample)
    shape_before = df.shape
    resampled = df.resample(frequency, kind='timestamp', base=base).bfill()
    
    if interpolate:
        resampled = resampled.interpolate(method=interpolate_method)    
        
    #print("dimention before resampling is: {}".format(shape_before))
    return resampled


# This function extracts timeseries of one named service from the whole raw timeseries data
def extract_metric_series(df, col_name, col_value):
    metric = df.loc[df[col_name] == col_value].drop([col_name], axis=1).rename(index=str, columns={"value": col_value})
    metric.sort_index(inplace=True)
    return metric  

# sum df rows, remove expanded columns and set a new column with a metric name
def sum_timeseries(df, columns_to_delete, metric_name, metric_col_name='metric'):
    df['value'] = df.sum(axis=1)
    df[metric_col_name] = metric_name
    for col in columns_to_delete:
        df = df.drop([col], axis=1)
    return df 

# max df rows, remove expanded columns and set a new column with a metric name
def max_timeseries(df, columns_to_delete, metric_name, metric_col_name='metric'):
    df['value'] = df.max(axis=1)
    df[metric_col_name] = metric_name
    for col in columns_to_delete:
        df = df.drop([col], axis=1)
    return df

# average df rows, remove expanded columns and set a new column with a metric name
def avg_timeseries(df, columns_to_delete, metric_name, metric_col_name='metric'):
    df['value'] = df.mean(axis=1)
    df[metric_col_name] = metric_name
    for col in columns_to_delete:
        df = df.drop([col], axis=1)
    return df

# average df rows, remove expanded columns and set a new column with a metric name
def count_timeseries(df, columns_to_delete, metric_name, metric_col_name='metric'):
    df['value'] = df.count(axis=1)
    df[metric_col_name] = metric_name
    for col in columns_to_delete:
        df = df.drop([col], axis=1)
    return df

def merge(df, series):
    return pd.merge_asof(df, series, left_index=True, right_index=True, tolerance=pd.Timedelta('1 second')).bfill()    

    

In [177]:
orders_ltcy_df = read_df(file_dir, orders_ltcy_file)
orders_ltcy_df = expand(orders_ltcy_df, base=37)
orders_ltcy_df.head(10) 

,orders_ltcy
date,
2020-03-26 19:47:07,0.240
2020-03-26 19:47:22,0.240
2020-03-26 19:47:37,0.240
2020-03-26 19:47:52,0.240
2020-03-26 19:48:07,0.240
2020-03-26 19:48:22,0.400
2020-03-26 19:48:37,0.390
2020-03-26 19:48:52,0.380
2020-03-26 19:49:07,0.240


In [178]:
carts_ltcy_df = read_df(file_dir, carts_ltcy_file)
carts_ltcy_df = expand(carts_ltcy_df, base=37)
carts_ltcy_df.head() 

,carts_ltcy
date,
2020-03-26 19:46:52,1.960
2020-03-26 19:47:07,0.980
2020-03-26 19:47:22,0.920
2020-03-26 19:47:37,0.490
2020-03-26 19:47:52,0.480


In [179]:
print('orders latency shape {} and carts latency shape {}'
       .format(orders_ltcy_df.shape,carts_ltcy_df.shape))


orders latency shape (6764, 1) and carts latency shape (6765, 1)


In [180]:
services_pods_df = read_df(file_dir, services_pods_file)
services_pods_df = expand(services_pods_df, base=37)
services_pods_df.head(10)  

,carts-db_pods,carts_pods,catalogue-db_pods,catalogue_pods,front-end_pods,orders-db_pods,orders_pods,payment_pods,queue-master_pods,rabbitmq_pods,session-db_pods,shipping_pods,user-db_pods,user_pods
date,,,,,,,,,,,,,,
2020-03-26 19:46:37,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.500,1.000,1.000,1.000,1.000
2020-03-26 19:46:52,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.500,1.000,1.000,1.000,1.000
2020-03-26 19:47:07,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.500,1.000,1.000,1.000,1.000
2020-03-26 19:47:22,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.500,1.000,1.000,1.000,1.000
2020-03-26 19:47:37,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.500,1.000,1.000,1.000,1.000
2020-03-26 19:47:52,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.500,1.000,1.000,1.000,1.000
2020-03-26 19:48:07,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.500,1.000,1.000,1.000,1.000
2020-03-26 19:48:22,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.500,1.000,1.000,1.000,1.000
2020-03-26 19:48:37,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.500,1.000,1.000,1.000,1.000


In [181]:
services_pods_df.shape


(6766, 14)

In [182]:
services_cpu_use_df = read_df(file_dir, service_cpu_usage_file)
services_cpu_use_df = expand(services_cpu_use_df, base=37)
services_cpu_use_df.head(10)    

,carts-db_cpu_use,carts_cpu_use,catalogue-db_cpu_use,catalogue_cpu_use,front-end_cpu_use,orders-db_cpu_use,orders_cpu_use,payment_cpu_use,queue-master_cpu_use,rabbitmq_cpu_use,session-db_cpu_use,shipping_cpu_use,user-db_cpu_use,user_cpu_use
date,,,,,,,,,,,,,,
2020-03-26 19:46:22,0.070,0.030,0.010,0.010,0.030,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
2020-03-26 19:46:37,0.180,0.180,0.010,0.020,0.060,0.020,0.020,0.010,0.020,0.020,0.020,0.010,0.020,0.020
2020-03-26 19:46:52,0.200,0.440,0.010,0.020,0.070,0.020,0.030,0.010,0.020,0.020,0.030,0.010,0.020,0.030
2020-03-26 19:47:07,0.170,0.230,0.010,0.020,0.070,0.020,0.030,0.010,0.020,0.020,0.030,0.010,0.020,0.020
2020-03-26 19:47:22,0.140,0.240,0.010,0.020,0.070,0.010,0.020,0.010,0.010,0.010,0.010,0.010,0.020,0.020
2020-03-26 19:47:37,0.180,0.180,0.010,0.020,0.070,0.010,0.030,0.010,0.010,0.010,0.010,0.010,0.020,0.020
2020-03-26 19:47:52,0.180,0.180,0.010,0.010,0.070,0.010,0.030,0.010,0.010,0.010,0.010,0.010,0.020,0.020
2020-03-26 19:48:07,0.170,0.130,0.010,0.010,0.060,0.010,0.020,0.010,0.010,0.010,0.010,0.010,0.020,0.020
2020-03-26 19:48:22,0.160,0.120,0.010,0.010,0.070,0.010,0.020,0.010,0.010,0.010,0.010,0.010,0.020,0.020


In [183]:
services_cpu_use_df.shape

(6766, 14)

In [184]:
nodes_cpu_use_df = read_df(file_dir, nodes_cpu_usage_file)
nodes_cpu_use_df = expand(nodes_cpu_use_df, base=37)
nodes_cpu_use_df.head()   

,nodes_cpu_use
date,
2020-03-26 19:46:22,2.500
2020-03-26 19:46:37,1.890
2020-03-26 19:46:52,1.350
2020-03-26 19:47:07,1.260
2020-03-26 19:47:22,1.280


In [185]:
orders_req_rate_df = read_df(file_dir, orders_req_rate_file)
orders_req_rate_df = expand(orders_req_rate_df, base=37)
#orders_req_rate_df = orders_req_rate_df[orders_req_rate_df.orders_req  > 0]
orders_req_rate_df.head()

,orders_req
date,
2020-03-26 19:46:52,0.150
2020-03-26 19:47:07,0.130
2020-03-26 19:47:22,0.360
2020-03-26 19:47:37,0.440
2020-03-26 19:47:52,0.440


In [186]:
carts_req_rate_df = read_df(file_dir, carts_req_rate_file)
carts_req_rate_df = expand(carts_req_rate_df, base=37)
#carts_req_rate_df = carts_req_rate_df[carts_req_rate_df.carts_req  > 0]
carts_req_rate_df.head()

,carts_req
date,
2020-03-26 19:46:37,0.470
2020-03-26 19:46:52,0.490
2020-03-26 19:47:07,1.040
2020-03-26 19:47:22,1.620
2020-03-26 19:47:37,1.780


# Orders Flow 

The features selected for the orders workflow are the requests rate, pod-level CPU utilization of the microservices including front-end, orders, users, shipping, payment, cart, users-db, orders- db, cart-db, and the CPU utilization the nodes that host these microservices

In [187]:
selected_services_cpu = ['front-end_cpu_use'
                     ,'orders_cpu_use'  
                     ,'orders-db_cpu_use'
                     ,'user_cpu_use'
                     ,'user-db_cpu_use'
                     ,'shipping_cpu_use'
                     ,'payment_cpu_use'
                     ,'carts_cpu_use'                                                                                
                     ,'carts-db_cpu_use'
                     ]
orders_flow_df = services_cpu_use_df[selected_services]

# add services pods
selected_services_cpu = ['front-end_pods'
                     ,'orders_pods'  
                     ,'orders-db_pods'
                     ,'user_pods'
                     ,'user-db_pods'
                     ,'shipping_pods'
                     ,'payment_pods'
                     ,'carts_pods'                                                                                
                     ,'carts-db_pods'
                     ]
orders_pods_df = services_pods_df[selected_services_cpu]
orders_flow_df = merge(orders_flow_df, orders_pods_df)

# add the nodes cpu use
orders_flow_df = merge(orders_flow_df, nodes_cpu_use_df)

# add the orders request rate
orders_flow_df = merge(orders_flow_df, orders_req_rate_df)

# add the orders latency target
orders_flow_df = merge(orders_flow_df, orders_ltcy_df)

orders_flow_df.head(5)



,front-end_cpu_use,orders_cpu_use,orders-db_cpu_use,user_cpu_use,user-db_cpu_use,shipping_cpu_use,payment_cpu_use,carts_cpu_use,carts-db_cpu_use,front-end_pods,...,orders-db_pods,user_pods,user-db_pods,shipping_pods,payment_pods,carts_pods,carts-db_pods,nodes_cpu_use,orders_req,orders_ltcy
date,,,,,,,,,,,,,,,,,,,,,
2020-03-26 19:46:22,0.030,0.010,0.010,0.010,0.010,0.010,0.010,0.030,0.070,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,2.500,0.150,0.240
2020-03-26 19:46:37,0.060,0.020,0.020,0.020,0.020,0.010,0.010,0.180,0.180,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.890,0.150,0.240
2020-03-26 19:46:52,0.070,0.030,0.020,0.030,0.020,0.010,0.010,0.440,0.200,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.350,0.150,0.240
2020-03-26 19:47:07,0.070,0.030,0.020,0.020,0.020,0.010,0.010,0.230,0.170,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.260,0.130,0.240
2020-03-26 19:47:22,0.070,0.020,0.010,0.020,0.020,0.010,0.010,0.240,0.140,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.280,0.360,0.240


In [188]:
orders_flow_df.shape

(6766, 21)

In [189]:
if 1 == 1:
    processed_file_path = processed_dir + 'orders_flow_data.csv'
    orders_flow_df.to_csv(path_or_buf=processed_file_path, index=True)

# Cart Flow

the features selected for the <b>cart workflow</b> are the request rate, the pod-level CPU utilization of the microservices including front-end, orders, cart, cart-db, and the CPU utilization  the nodes that host these microservices.

In [190]:
selected_services = ['front-end_cpu_use'
                     ,'orders_cpu_use'                                                           
                     ,'carts_cpu_use'                                                                                
                     ,'carts-db_cpu_use'
                     ]
carts_flow_df = services_cpu_use_df[selected_services]

# add the nodes cpu use
carts_flow_df = merge(carts_flow_df, nodes_cpu_use_df)

# add the orders request rate
carts_flow_df = merge(carts_flow_df, carts_req_rate_df)

# add the orders latency target
carts_flow_df = merge(carts_flow_df, carts_ltcy_df)

carts_flow_df.head(5)


,front-end_cpu_use,orders_cpu_use,carts_cpu_use,carts-db_cpu_use,nodes_cpu_use,carts_req,carts_ltcy
date,,,,,,,
2020-03-26 19:46:22,0.030,0.010,0.030,0.070,2.500,0.470,1.960
2020-03-26 19:46:37,0.060,0.020,0.180,0.180,1.890,0.470,1.960
2020-03-26 19:46:52,0.070,0.030,0.440,0.200,1.350,0.490,1.960
2020-03-26 19:47:07,0.070,0.030,0.230,0.170,1.260,1.040,0.980
2020-03-26 19:47:22,0.070,0.020,0.240,0.140,1.280,1.620,0.920


In [191]:
carts_flow_df.shape

(6766, 7)

In [192]:
if 1 == 0:
    processed_file_path = processed_dir + 'cart_flow_data.csv'
    carts_flow_df.to_csv(path_or_buf=processed_file_path, index=True)